### Load Libraries

In [22]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import LSTM, SpatialDropout1D 
from sklearn.model_selection import train_test_split
import csv
import keras
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import os

### Load Raw Training Data

In [2]:
inpData = []
labels = []
with open('cdata.csv') as csv_file:
    lines = csv.reader(csv_file, delimiter=',')
    for row in lines:
        labels.append(row[0])
        inpData.append(float(row[1]))
        

### Set the Target Labels

In [3]:
minThreshold = 30
maxThreshold = 70
target = []
index = 0
while index + 1 < len(inpData):
    if inpData[index + 1] > maxThreshold:# Scale Up
        target.append(1)
    elif inpData[index + 1] < minThreshold: # Scale Down
        target.append(-1)
    else:
        target.append(0)  #Remain same
    index += 1

In [ ]:
print(target)

### Preprocessing Input Data

In [4]:
cpuUsage = []
cpuScaling = []
historyParam = 50
index = historyParam

while index<len(inpData):
    currRec = [[inpData[i]] for i in range(index-historyParam,index)]
    cpuUsage.append(currRec)
    index+=1
    

In [5]:
cpuScaling = target[historyParam-1:len(target)]


cpuUsageData = np.array(cpuUsage,dtype = 'float')
cpuScaling = np.array(cpuScaling,dtype = 'float')

cpuScalingCategorical = to_categorical(cpuScaling, num_classes = 3)

In [6]:
#len(inpData)
X_train, X_test, y_train, y_test = train_test_split(cpuUsageData , cpuScalingCategorical, test_size=0.25, random_state=42)

In [ ]:
cpuUsageData.shape
#np.shape(cpuUsage)

In [55]:
import pandas
testrec = 5
print(cpuScalingCategorical[testrec])
print(cpuScaling[testrec])
print(set(cpuScaling))
myseries=pandas.Series(cpuScaling)
print(myseries.value_counts())

[0. 0. 1.]
-1.0
set([0.0, 1.0, -1.0])
-1.0    4121
 0.0     523
 1.0     391
dtype: int64


In [7]:
model = Sequential()
#model.add(SpatialDropout1D(0.3))
model.add(LSTM((50),dropout = 0.3,batch_input_shape = (None,50,1), return_sequences = True))
model.add(Dense(50, activation = 'tanh'))
#model.add(SpatialDropout1D(0.3))
model.add(LSTM(25,return_sequences = False))
model.add(Dense(3, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam',metrics = ['accuracy'])
#model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [16]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 50, 50)            10400     
_________________________________________________________________
dense_1 (Dense)              (None, 50, 50)            2550      
_________________________________________________________________
lstm_2 (LSTM)                (None, 25)                7600      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 78        
Total params: 20,628
Trainable params: 20,628
Non-trainable params: 0
_________________________________________________________________


In [19]:
filepath="LSTMBestWeights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [23]:
if os.path.isfile("LSTMBestWeights.hdf5"):
    #print("Hey")
    model.load_weights("LSTMBestWeights.hdf5")

Hey


In [20]:
history = model.fit(X_train, y_train, epochs = 50,validation_data = [X_test, y_test], callbacks = callbacks_list,verbose=0)

Instructions for updating:
Use tf.cast instead.

Epoch 00001: val_acc improved from -inf to 0.85703, saving model to LSTMBestWeights.hdf5

Epoch 00002: val_acc improved from 0.85703 to 0.86021, saving model to LSTMBestWeights.hdf5

Epoch 00003: val_acc improved from 0.86021 to 0.89913, saving model to LSTMBestWeights.hdf5

Epoch 00004: val_acc improved from 0.89913 to 0.92851, saving model to LSTMBestWeights.hdf5

Epoch 00005: val_acc did not improve from 0.92851

Epoch 00006: val_acc improved from 0.92851 to 0.93169, saving model to LSTMBestWeights.hdf5

Epoch 00007: val_acc did not improve from 0.93169

Epoch 00008: val_acc did not improve from 0.93169

Epoch 00009: val_acc did not improve from 0.93169

Epoch 00010: val_acc did not improve from 0.93169

Epoch 00011: val_acc did not improve from 0.93169

Epoch 00012: val_acc did not improve from 0.93169

Epoch 00013: val_acc did not improve from 0.93169

Epoch 00014: val_acc did not improve from 0.93169

Epoch 00015: val_acc did not i

In [33]:
X_test.shape
np.expand_dims(X_test[0],axis = 0).shape

(1, 50, 1)

In [37]:
res = model.predict(np.expand_dims(X_test[0],axis=0))

In [38]:
print(res)
print(np.argmax(res[0]))

[[7.9184992e-04 6.0061505e-04 9.9860758e-01]]
2


In [42]:
np.argmax([0,1,0],axis = 0)

1

In [ ]:
np.argmax(res,axis = 1)

In [ ]:
print(inpData[0+1])